# 2RP Net - Teste para Data Engineer
### Desafio:
### - 2.	Realize a extração dos dados dos 3 ultimos meses de prescrição

In [22]:
# Para essa parte do desafio escolheu-se trabalhar com o consumo de API
# Para tanto, foi utilizado como Linguagem padrão o Python - Jupyter e como Banco de Dados o MongoDB
# Já para o consumo da API em si, foi necessário a utilização das bibliotecas:
# Requests - Para solicitações HTTP
# json - Para trabalhar com arquivos JSON
# pymongo - Para realizar a conexão entre o Python e o Banco de Dados o MongoDB
# bson.son.- Usado principalmente no pymongo, o qual será importante na persistência e tratamento dos dados

import json
import requests
from pymongo import MongoClient
from ast import main
from bson.son import SON


In [23]:
#Funções em Python
#Função criada para inserir os dados

def inserir_dados(client, collection, dados):
    collection.insert_many(dados)
    client.close()
    print("Dados Inseridos com Sucesso")

In [24]:
#Função elaborada para a criação do banco de dado contendo o database e a collection a ser trabalhado

def criar_banco(dados, nome_tabela):
    client = MongoClient('localhost', 27017)
    db = client['banco']
    collection = db[nome_tabela]
    inserir_dados(client, collection, dados)

In [25]:
# Como o desafio pede para separar os dados entre prescribers e prescriptions
# Foi criado uma função para consumo e armazenamento dos dados para cada situação
# Aqui temos o consumo da API - prescribers (Prescritores)
# Sendo estabelecida a condição do desafio "extração dos dados dos 3 ultimos meses"
# Como o enunciado possibiltou o trabalho com API limitando a quantidade, 
# Escolheu-se o intervalo de 10mil linhas de coletas da API prescribers
# De um "total": 17621811 para o mês 08
# De um "total": 17604125 para o mês 07
# De um "total": 17603900 para o mês 06

def requisicao_epd():
    for mes in range(8, 5, -1):
        api = requests.get(f'https://opendata.nhsbsa.net/api/3/action/datastore_search?resource_id=EPD_20220{mes}&limit=10000'.format(mes))
        dados = api.json()['result']['records']
        criar_banco(dados, 'tabela1')


        

In [26]:
# Aqui temos o consumo da API -  prescriptions (prescrições)
# Aqui escolhe-se trablhar com um função que realiza a extração dos dados dos 3 ultimos anos"
# Como o enunciado possibiltou o trabalho com API limitando a sua quantidade, 
# Escolheu-se o intervalo de 10mil linhas de coletas da API prescriptions
# De um "total": 752697 para o ano de 2021
# De um "total": 768231 para o ano de 2020
# De um "total": 802855 para o ano de 2019

def requisicao_pca():
    for ano in range(20, 17, -1):
        api = requests.get(f'https://opendata.nhsbsa.net/api/3/action/datastore_search?resource_id=PCA_STP_SNOMED_20{ano}&limit=10000'.format(ano))
        dados = api.json()['result']['records']
        criar_banco(dados, 'tabela2')
        

In [27]:
if __name__ == '__main__':
    print("")
    print("Inserção dos dados - Prescrições")
    requisicao_pca()
    
    print("")
    print("Inserção dos dados - Prescritores")
    requisicao_epd()
    
    print("")
    print("Processo Finalizado com Sucesso\n")


Inserção dos dados - Prescrições
Dados Inseridos com Sucesso
Dados Inseridos com Sucesso
Dados Inseridos com Sucesso

Inserção dos dados - Prescritores
Dados Inseridos com Sucesso
Dados Inseridos com Sucesso
Dados Inseridos com Sucesso

Processo Finalizado com Sucesso



### 6 - Gere scripts que atendam as solicitações abaixo:

#### - Para a realização dessa etapa do desavio, foi imprescindívell a utilização  do "Studio 3T for MongoDB", o qual além de facilitar a visualização da conexão com o banco Mongo, possibilitou a transposição de uma linguagem SQL ou NoSQL para comandos Python.

In [28]:
# A) Crie um dataframe contendo os 10 principais produtos químicos prescritos por região.

from bson.son import SON
from pymongo import MongoClient


#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela2"]

#Lista contendo o nome de todas as regiões
lista = ['SOUTH EAST','MIDLANDS AND EAST OF ENGLAND (C 01-Apr-20)',
         'SOUTH OF ENGLAND (C 01-Apr-20)', 'SOUTH WEST', 'MIDLANDS', 
         'LONDON', 'NORTH EAST AND YORKSHIRE' ,'NORTH WEST', 'EAST OF ENGLAND' ]

for i in range(len(lista)):
    pipeline = [
        {
            u"$match": {
                u"REGION_NAME": lista[i]
            }
        }, 
        {
            u"$group": {
                u"_id": {
                    u"BNF_SECTION": u"$BNF_SECTION"
                },
                u"COUNT(BNF_SECTION)": {
                    u"$sum": 1
                }
            }
        }, 
        {
            u"$project": {
                u"COUNT(BNF_SECTION)": u"$COUNT(BNF_SECTION)",
                u"BNF_SECTION": u"$_id.BNF_SECTION",
                u"_id": 0
            }
        }, 
        {
            u"$sort": SON([ (u"COUNT(BNF_SECTION)", -1) ])
        }, 
        {
            u"$limit": 10
        }
    ]

    cursor = collection.aggregate(
        pipeline, 
        allowDiskUse = True
    )
    
    print("")
    print(lista[i])
    print("")
    
    for doc in cursor:

        print(doc)

    
        
client.close()


SOUTH EAST

{'COUNT(BNF_SECTION)': 857, 'BNF_SECTION': 'Beta-adrenoceptor blocking drugs'}
{'COUNT(BNF_SECTION)': 728, 'BNF_SECTION': 'Hypertension and heart failure'}
{'COUNT(BNF_SECTION)': 721, 'BNF_SECTION': 'Antisecretory drugs and mucosal protectants'}
{'COUNT(BNF_SECTION)': 583, 'BNF_SECTION': 'Diuretics'}
{'COUNT(BNF_SECTION)': 459, 'BNF_SECTION': 'Laxatives'}
{'COUNT(BNF_SECTION)': 286, 'BNF_SECTION': 'Antispasmodics and other drugs altering gut motility'}
{'COUNT(BNF_SECTION)': 210, 'BNF_SECTION': 'Chronic bowel disorders'}
{'COUNT(BNF_SECTION)': 162, 'BNF_SECTION': 'Anti-arrhythmic drugs'}
{'COUNT(BNF_SECTION)': 85, 'BNF_SECTION': 'Dyspepsia and gastro-oesophageal reflux disease'}
{'COUNT(BNF_SECTION)': 54, 'BNF_SECTION': 'Drugs affecting intestinal secretions'}

MIDLANDS AND EAST OF ENGLAND (C 01-Apr-20)

{'COUNT(BNF_SECTION)': 41, 'BNF_SECTION': 'Beta-adrenoceptor blocking drugs'}
{'COUNT(BNF_SECTION)': 39, 'BNF_SECTION': 'Antisecretory drugs and mucosal protectants'}
{'CO

In [29]:
# 6) B - Quais produtos químicos prescritos tiveram a maior somatória de custos por mês? Liste os 10 primeiros.
from bson.son import SON
from pymongo import MongoClient

#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela1"]


#Lista contendo o nome dos meses a serem consultados 
lista = [202207, 202208, 202206]

for i in range(len(lista)):
    pipeline = [
    {
        u"$match": {
            u"YEAR_MONTH": (lista[i])
        }
    }, 
    {
        u"$group": {
            u"_id": {
                u"YEAR_MONTH": u"$YEAR_MONTH",
                u"ICB_CODE": u"$ICB_CODE"
            },
            u"SUM(ACTUAL_COST)": {
                u"$sum": u"$ACTUAL_COST"
            }
        }
    }, 
    {
        u"$project": {
            u"SUM(ACTUAL_COST)": u"$SUM(ACTUAL_COST)",
            u"YEAR_MONTH": u"$_id.YEAR_MONTH",
            u"_id": 0
        }
    }, 
    {
        u"$sort": SON([ (u"YEAR_MONTH", 1) ])
    }, 
    {
        u"$limit": 10
    }
    ]

    cursor = collection.aggregate(
        pipeline, 
        allowDiskUse = True
    )
    
    print("")
    print(lista[i])
    print("")
    
    for doc in cursor:
        print(doc)

    
        
client.close()


202207

{'SUM(ACTUAL_COST)': 82951.40002, 'YEAR_MONTH': 202207}
{'SUM(ACTUAL_COST)': 7737.28457, 'YEAR_MONTH': 202207}
{'SUM(ACTUAL_COST)': 122437.12867, 'YEAR_MONTH': 202207}
{'SUM(ACTUAL_COST)': 227634.22391, 'YEAR_MONTH': 202207}

202208

{'SUM(ACTUAL_COST)': 3755.48009, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 663.08682, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 7106.92411, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 579.9309, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 1481.97098, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 1023.38472, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 130.98767999999998, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 430.36577, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 103464.53441, 'YEAR_MONTH': 202208}
{'SUM(ACTUAL_COST)': 650.35332, 'YEAR_MONTH': 202208}

202206

{'SUM(ACTUAL_COST)': 9920.18462, 'YEAR_MONTH': 202206}
{'SUM(ACTUAL_COST)': 39.80944, 'YEAR_MONTH': 202206}
{'SUM(ACTUAL_COST)': 1127.5845199999999, 'YEAR_MONTH': 202206}
{'SUM(ACTUAL_COST)': 4

In [30]:
# 6) C - Quais são as precrições mais comuns?

#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela2"]

pipeline = [
    {
        u"$group": {
            u"_id": {
                u"BNF_PARAGRAPH": u"$BNF_PARAGRAPH"
            },
            u"COUNT(PRESCRIBED_PREP_CLASS)": {
                u"$sum": 1
            }
        }
    }, 
    {
        u"$project": {
            u"COUNT(PRESCRIBED_PREP_CLASS)": u"$COUNT(PRESCRIBED_PREP_CLASS)",
            u"BNF_PARAGRAPH": u"$_id.BNF_PARAGRAPH",
            u"_id": 0
        }
    }, 
    {
        u"$sort": SON([ (u"COUNT(PRESCRIBED_PREP_CLASS)", -1) ])
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = True
)
try:
    for doc in cursor:
        print(doc)
finally:
    client.close()

{'COUNT(PRESCRIBED_PREP_CLASS)': 6004, 'BNF_PARAGRAPH': 'Beta-adrenoceptor blocking drugs'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 2963, 'BNF_PARAGRAPH': 'Proton pump inhibitors'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 2541, 'BNF_PARAGRAPH': 'Renin-angiotensin system drugs'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 2110, 'BNF_PARAGRAPH': 'H2-receptor antagonists'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1981, 'BNF_PARAGRAPH': 'Antispasmodics and other drugs altering gut motility'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1936, 'BNF_PARAGRAPH': 'Stimulant laxatives'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1619, 'BNF_PARAGRAPH': 'Loop diuretics'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1312, 'BNF_PARAGRAPH': 'Alpha-adrenoceptor blocking drugs'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1138, 'BNF_PARAGRAPH': 'Drugs for arrhythmias'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 1056, 'BNF_PARAGRAPH': 'Aminosalicylates'}
{'COUNT(PRESCRIBED_PREP_CLASS)': 970, 'BNF_PARAGRAPH': 'Potassium-sparing diuretics and aldosterone antagonists'}
{'COUNT(PRESCRIBED_PREP_

In [34]:
# 6) D - Qual produto químico é mais prescrito por cada prescriber?

from bson.son import SON
from pymongo import MongoClient

#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela1"]

# Aqui é listado o nome do prescriber/ Quantidade do Produto mais prescrito, Nome do Produto mais prescrito 
pipeline = [
    {
        u"$group": {
            u"_id": {
                u"CHEMICAL_SUBSTANCE_BNF_DESCR": u"$CHEMICAL_SUBSTANCE_BNF_DESCR",
                u"PRACTICE_NAME": u"$PRACTICE_NAME"
            },
            u"COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)": {
                u"$sum": 1
            }
        }
    }, 
    {
        u"$project": {
            u"PRACTICE_NAME": u"$_id.PRACTICE_NAME",
            u"COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)": u"$COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)",
            u"CHEMICAL_SUBSTANCE_BNF_DESCR": u"$_id.CHEMICAL_SUBSTANCE_BNF_DESCR",
            u"_id": 0
        }
    }, 
    {
        u"$group": {
            u"_id": None,
            u"distinct": {
                u"$addToSet": u"$$ROOT"
            }
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$distinct",
            u"preserveNullAndEmptyArrays": False
        }
    }, 
    {
        u"$replaceRoot": {
            u"newRoot": u"$distinct"
        }
    }, 
    {
        u"$sort": SON([ (u"COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)", -1) ])
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = True
)
try:
    for doc in cursor:
        print(doc)
finally:
    client.close()

{'PRACTICE_NAME': 'UNIDENTIFIED DOCTORS', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 31, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Wound Management & Other Dressings'}
{'PRACTICE_NAME': 'UNIDENTIFIED DOCTORS', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 26, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Catheters'}
{'PRACTICE_NAME': 'UNIDENTIFIED DOCTORS', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 17, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Methadone hydrochloride'}
{'PRACTICE_NAME': 'TURNING POINT - HEREFORDSHIRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 16, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Methadone hydrochloride'}
{'PRACTICE_NAME': 'ADDICTIONS SERVICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 15, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Methadone hydrochloride'}
{'PRACTICE_NAME': 'WE ARE WITH YOU NORTH LINCOLNSHIRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 14, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Methadone hydrochloride'}
{'PRACTICE_NAME': 'ANCORA MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 12, 'CHEMICAL_SUBSTANCE_BNF_DESCR'

{'PRACTICE_NAME': 'HELMSLEY SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Nicorandil'}
{'PRACTICE_NAME': 'THE KIRTON LINDSEY AND SCOTTER SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Cyanocobalamin'}
{'PRACTICE_NAME': 'NORTH SHORE SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Zolmitriptan'}
{'PRACTICE_NAME': 'PEMBROKE MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Adhesive Removers (Sprays/Liquids/Wipes)'}
{'PRACTICE_NAME': 'FORD MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Omeprazole'}
{'PRACTICE_NAME': 'HAXBY GROUP PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Zopiclone'}
{'PRACTICE_NAME': 'SUNNYSIDE MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Clonazepam'}
{'PRACTICE_NAME': 'WINTERTON MEDICA

{'PRACTICE_NAME': 'KINGSTEIGNTON MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Sertraline hydrochloride'}
{'PRACTICE_NAME': 'JOSHI NA', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Co-codamol (Codeine phosphate/paracetamol)'}
{'PRACTICE_NAME': 'THORPE HESLEY SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Carbocisteine'}
{'PRACTICE_NAME': 'OAKENHURST MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Enteral nutrition'}
{'PRACTICE_NAME': 'PEEL GPS', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Atorvastatin'}
{'PRACTICE_NAME': 'SOUTH BRENT HEALTH CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Clotrimazole'}
{'PRACTICE_NAME': 'WYCLIFFE SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 2, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Catheters'}
{'PRACTICE_NAME': "ST. LAURENCE'S ME

{'PRACTICE_NAME': 'BURN BRAE MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Ivermectin'}
{'PRACTICE_NAME': 'FELTON SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Sertraline hydrochloride'}
{'PRACTICE_NAME': 'THE AVENUE SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Aspirin'}
{'PRACTICE_NAME': 'OLD FARM SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Macrogol 3350'}
{'PRACTICE_NAME': 'WINGATE MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Candesartan cilexetil'}
{'PRACTICE_NAME': 'CROMPTON VIEW SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Docusate sodium'}
{'PRACTICE_NAME': 'THE BANKS & BEARWOOD MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Clobazam'}
{'PRACTICE_NAME': 'COLLIERY COURT MEDICAL GROUP', 'CO

{'PRACTICE_NAME': 'ASHBURN MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Zopiclone'}
{'PRACTICE_NAME': 'THE BRIDGES MEDICAL CTR.', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Tadalafil'}
{'PRACTICE_NAME': 'BRIDGE VIEW MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Brinzolamide/brimonidine'}
{'PRACTICE_NAME': 'WHIPTON SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Citalopram hydrobromide'}
{'PRACTICE_NAME': 'KEARSLEY MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Nortriptyline'}
{'PRACTICE_NAME': 'MAYFLOWER MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Lithium carbonate'}
{'PRACTICE_NAME': 'ST GEORGES SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Clindamycin/tretinoin'}
{'PRACTICE_NAME': 'PRIMROSE BANK M

...

{'PRACTICE_NAME': 'MARINE MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Beclometdiprop/formoterol/glycopyrronium'}
{'PRACTICE_NAME': 'MAYFIELD MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Tibolone'}
{'PRACTICE_NAME': 'DEANE MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Donepezil hydrochloride'}
{'PRACTICE_NAME': 'LEATSIDE SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Celecoxib'}
{'PRACTICE_NAME': 'BRANNAM MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Skin Fillers And Protectives'}
{'PRACTICE_NAME': 'STOCKTON HEATH MED.CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Lansoprazole'}
{'PRACTICE_NAME': 'REDLANDS PRIMARY CARE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Betahistine hydrochloride'}
{'PRACTI

{'PRACTICE_NAME': 'THE KIRTON LINDSEY AND SCOTTER SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Pancreatin'}
{'PRACTICE_NAME': 'FREMINGTON MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Verapamil hydrochloride'}
{'PRACTICE_NAME': 'CONCORD MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Bag Covers'}
{'PRACTICE_NAME': 'BEAUFORT ROAD SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Latanoprost'}
{'PRACTICE_NAME': 'ELM SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Levomepromazine hydrochloride'}
{'PRACTICE_NAME': 'ST NEOTS SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Potassium chloride'}
{'PRACTICE_NAME': 'FIRST CONTACT & PC PATH', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Desmopressin acetate'}
{'PRACTICE_NAME': 'BA

{'PRACTICE_NAME': 'WEMBURY SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Bimatoprost'}
{'PRACTICE_NAME': 'ALNWICK MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Sildenafil (Erectile Dysfunction)'}
{'PRACTICE_NAME': 'THE CENTRAL SURGERY BARTON', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Aripiprazole'}
{'PRACTICE_NAME': 'IMPERIAL SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Ispaghula husk'}
{'PRACTICE_NAME': 'SUNDERLAND EXTENDED ACCESS SERVICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Diazepam'}
{'PRACTICE_NAME': 'MOUNT PLEASANT HEALTH CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Oxycodone hydrochloride'}
{'PRACTICE_NAME': 'JOSHI NA', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Budesonide'}
{'PRACTICE_NAME': 'THE LYNTON PRACTICE', 

{'PRACTICE_NAME': 'CARE PLUS GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Physical Debridement Device'}
{'PRACTICE_NAME': 'RIVERSIDE SURGERY', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Lymecycline'}
{'PRACTICE_NAME': 'MID DEVON MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Pravastatin sodium'}
{'PRACTICE_NAME': 'CHANNEL VIEW MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Other Appliances'}
{'PRACTICE_NAME': 'TRINITY MEDICAL CENTRE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Other Appliances'}
{'PRACTICE_NAME': 'ST THOMAS MEDICAL GROUP', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Oxybutynin'}
{'PRACTICE_NAME': 'CONCORD MEDICAL PRACTICE', 'COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR)': 1, 'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Macrogol 3350'}
{'PRACTICE_NAME': 'SOUTHOVER MEDICAL

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
# 6) E - Quantos prescribers foram adicionados no ultimo mês? 
from bson.int64 import Int64
from pymongo import MongoClient

#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela1"]

#Aqui seria possivel criar uma parametrização para contemplar os diferentes meses
# Mas como foi pedido apenas referente ao ultimo mês. 
pipeline = [
    {
        u"$match": {
            u"YEAR_MONTH": Int64(202208)
        }
    }, 
    {
        u"$project": {
            u"CHEMICAL_SUBSTANCE_BNF_DESCR": u"$CHEMICAL_SUBSTANCE_BNF_DESCR",
            u"YEAR_MONTH": u"$YEAR_MONTH",
            u"_id": 0
        }
    }, 
    {
        u"$group": {
            u"_id": None,
            u"distinct": {
                u"$addToSet": u"$$ROOT"
            }
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$distinct",
            u"preserveNullAndEmptyArrays": False
        }
    }, 
    {
        u"$replaceRoot": {
            u"newRoot": u"$distinct"
        }
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = True
)
try:
    for doc in cursor:
        print(doc)
finally:
    client.close()

{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Aclidinium bromide', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Fluticasone propionate (Inhalation)', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Atenolol', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Methylprednisolone acetate', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Ketorolac trometamol', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Fluconazole', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Other sunscreening preparations', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Midodrine hydrochloride', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Eye Products', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Hydralazine hydrochloride', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Ospemifene', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Timothy grass pollen allergen extract', 'YEAR_MONTH': 202208}
{'CHEMICAL_SUBSTANCE_BNF_DESCR': 'Di

In [36]:
# 6) F - Quais prescribers atuam em mais de uma região? 

from bson.int64 import Int64
from bson.son import SON
from pymongo import MongoClient

#Realizando a conexão com o banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela1"]

pipeline = [
    {
        u"$group": {
            u"_id": {
                u"PRACTICE_NAME": u"$PRACTICE_NAME"
            },
            u"COUNT(REGIONAL_OFFICE_NAME)": {
                u"$sum": 1
            }
        }
    }, 
    {
        u"$project": {
            u"PRACTICE_NAME": u"$_id.PRACTICE_NAME",
            u"COUNT(REGIONAL_OFFICE_NAME)": u"$COUNT(REGIONAL_OFFICE_NAME)",
            u"_id": 0
        }
    }, 
    {
        u"$match": {
            u"COUNT(REGIONAL_OFFICE_NAME)": {
                u"$gt": Int64(1)
            }
        }
    }, 
    {
        u"$sort": SON([ (u"COUNT(REGIONAL_OFFICE_NAME)", -1) ])
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = True
)
try:
    for doc in cursor:
        print(doc)
finally:
    client.close()

{'PRACTICE_NAME': 'UNIDENTIFIED DOCTORS', 'COUNT(REGIONAL_OFFICE_NAME)': 650}
{'PRACTICE_NAME': 'HAXBY GROUP PRACTICE', 'COUNT(REGIONAL_OFFICE_NAME)': 349}
{'PRACTICE_NAME': 'ANCORA MEDICAL PRACTICE', 'COUNT(REGIONAL_OFFICE_NAME)': 294}
{'PRACTICE_NAME': 'CLEE MEDICAL CENTRE', 'COUNT(REGIONAL_OFFICE_NAME)': 279}
{'PRACTICE_NAME': 'DINNINGTON GROUP PRACTICE', 'COUNT(REGIONAL_OFFICE_NAME)': 277}
{'PRACTICE_NAME': 'WOODA SURGERY', 'COUNT(REGIONAL_OFFICE_NAME)': 271}
{'PRACTICE_NAME': 'THE CENTRAL SURGERY BARTON', 'COUNT(REGIONAL_OFFICE_NAME)': 257}
{'PRACTICE_NAME': 'RIVERSIDE SURGERY', 'COUNT(REGIONAL_OFFICE_NAME)': 228}
{'PRACTICE_NAME': 'WINGATE MEDICAL CENTRE', 'COUNT(REGIONAL_OFFICE_NAME)': 222}
{'PRACTICE_NAME': 'SPRING HALL GROUP PRACTICE', 'COUNT(REGIONAL_OFFICE_NAME)': 218}
{'PRACTICE_NAME': 'BEACON MEDICAL', 'COUNT(REGIONAL_OFFICE_NAME)': 213}
{'PRACTICE_NAME': 'STOCKTON HEATH MED.CENTRE', 'COUNT(REGIONAL_OFFICE_NAME)': 208}
{'PRACTICE_NAME': 'SUNNYSIDE MEDICAL CENTRE', 'COUNT(R

In [42]:
# 6) G - Qual o preço médio dos químicos prescritos em no ultimo mês coletado?

from bson.int64 import Int64
from pymongo import MongoClient

#Realizando a conexão com o Banco
client = MongoClient('localhost', 27017)
database = client["banco"]
collection = database["tabela1"]

print("Mês - 08")
pipeline = [
    {
        u"$match": {
            u"YEAR_MONTH": Int64(202208)
        }
    }, 
    {
        u"$group": {
            u"_id": {},
            u"AVG(ACTUAL_COST)": {
                u"$avg": u"$ACTUAL_COST"
            }
        }
    }, 
    {
        u"$project": {
            u"AVG(ACTUAL_COST)": u"$AVG(ACTUAL_COST)",
            u"_id": 0
        }
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = True
)
try:
    for doc in cursor:
        print(doc)
finally:
    client.close()

Mês - 08
{'AVG(ACTUAL_COST)': 48.528128153}


In [ ]:
# G) H - Gere uma tabela que contenha apenas a prescrição de maior valor de cada usuário.
# Nesse exercício não foi possivel entender o que o enunciado quis dizer com o usuário 